In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers seqeval matplotlib seaborn pandas sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=6600f887cb312d538432a14129c8e0a86a7be622b852cfcae7a31061da487652
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0

In [3]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("surrey-nlp/PLOD-CW-25")

# Convert to DataFrame
train_df = dataset["train"].to_pandas()
val_df = dataset["validation"].to_pandas()
test_df = dataset["test"].to_pandas()

# Show dataset info
print("Dataset Sizes:")
print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

# Preview dataset
train_df.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/268 [00:00<?, ?B/s]

PLOD-CW-25-Train.parquet:   0%|          | 0.00/343k [00:00<?, ?B/s]

PLOD-CW-25-Test.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

PLOD-CW-25-Val.parquet:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/150 [00:00<?, ? examples/s]

Dataset Sizes:
Train: 2000, Validation: 150, Test: 250


,tokens,pos_tags,ner_tags
0,"[MRR, ,, mortality, rate, ratio, ;, TBI, ,, tr...","[NOUN, PUNCT, NOUN, NOUN, NOUN, PUNCT, PROPN, ...","[B-AC, O, B-LF, I-LF, I-LF, O, B-AC, O, B-LF, ..."
1,"[HPC, ,, hippocampus, ;, mOFC, ,, medial, OFC,...","[PROPN, PUNCT, NOUN, PUNCT, NOUN, PUNCT, ADJ, ...","[B-AC, O, B-LF, O, O, O, O, B-AC, O, B-AC, O, ..."
2,"[In, addition, to, the, effects, on, cell, gro...","[ADP, NOUN, ADP, DET, NOUN, ADP, NOUN, NOUN, C...","[O, O, O, O, O, O, O, O, O, O, O, O, B-AC, B-A..."
3,"[Abbreviations, :, HC, ,, C, -, terminal, rece...","[NOUN, PUNCT, PROPN, PUNCT, NOUN, PUNCT, ADJ, ...","[O, O, B-AC, O, B-LF, I-LF, I-LF, I-LF, I-LF, ..."
4,"[LL, :, lower, limit, ,, UL, :, upper, limit, .]","[PROPN, PUNCT, ADJ, NOUN, PUNCT, PROPN, PUNCT,...","[B-AC, O, B-LF, I-LF, O, B-AC, O, B-LF, I-LF, O]"


In [4]:
# Check for missing values
print(train_df.isnull().sum())
print(val_df.isnull().sum())
print(test_df.isnull().sum())

tokens      0
pos_tags    0
ner_tags    0
dtype: int64
tokens      0
pos_tags    0
ner_tags    0
dtype: int64
tokens      0
pos_tags    0
ner_tags    0
dtype: int64


In [5]:
import re
# Clean text function
def clean_text(text_list):
    return [re.sub(r"[^a-zA-Z0-9\s]", "", token) for token in text_list]
# Remove empty token function
def remove_empty_tokens(tokens, pos_tags, ner_tags):
    new_tokens, new_pos, new_ner = [], [], []
    for token, pos, ner in zip(tokens, pos_tags, ner_tags):
        if token.strip() != "":
            new_tokens.append(token)
            new_pos.append(pos)
            new_ner.append(ner)
    return new_tokens, new_pos, new_ner

In [6]:
import pandas as pd
# Apply cleaning to tokens
train_df["tokens"] = train_df["tokens"].apply(clean_text)
val_df["tokens"] = val_df["tokens"].apply(clean_text)
test_df["tokens"] = test_df["tokens"].apply(clean_text)
# Remove empty tokens
train_df[["tokens", "pos_tags", "ner_tags"]] = train_df.apply(
    lambda row: pd.Series(remove_empty_tokens(row["tokens"], row["pos_tags"], row["ner_tags"])),
    axis=1
)
val_df[["tokens", "pos_tags", "ner_tags"]] = val_df.apply(
    lambda row: pd.Series(remove_empty_tokens(row["tokens"], row["pos_tags"], row["ner_tags"])),
    axis=1
)
test_df[["tokens", "pos_tags", "ner_tags"]] = test_df.apply(
    lambda row: pd.Series(remove_empty_tokens(row["tokens"], row["pos_tags"], row["ner_tags"])),
    axis=1
)
print("Text cleaned and empty tokens removed.")
train_df.head()

Text cleaned and empty tokens removed.


,tokens,pos_tags,ner_tags
0,"[MRR, mortality, rate, ratio, TBI, traumatic, ...","[NOUN, NOUN, NOUN, NOUN, PROPN, ADJ, NOUN, NOUN]","[B-AC, B-LF, I-LF, I-LF, B-AC, B-LF, I-LF, I-LF]"
1,"[HPC, hippocampus, mOFC, medial, OFC, OFC, orb...","[PROPN, NOUN, NOUN, ADJ, PROPN, PROPN, ADJ, NO...","[B-AC, B-LF, O, O, B-AC, B-AC, B-LF, I-LF, B-A..."
2,"[In, addition, to, the, effects, on, cell, gro...","[ADP, NOUN, ADP, DET, NOUN, ADP, NOUN, NOUN, C...","[O, O, O, O, O, O, O, O, O, O, O, B-AC, B-AC, ..."
3,"[Abbreviations, HC, C, terminal, receptor, bin...","[NOUN, PROPN, NOUN, ADJ, NOUN, VERB, NOUN, NOU...","[O, B-AC, B-LF, I-LF, I-LF, I-LF, I-LF, B-AC, ..."
4,"[LL, lower, limit, UL, upper, limit]","[PROPN, ADJ, NOUN, PROPN, ADJ, NOUN]","[B-AC, B-LF, I-LF, B-AC, B-LF, I-LF]"


In [11]:
def extract_features(sentence, pos_tags):
    features = []
    for i, word in enumerate(sentence):
        word_features = {
            "word": word,
            "is_upper": word.isupper(),
            "is_title": word.istitle(),
            "is_digit": word.isdigit(),
            "prefix-3": word[:3],
            "suffix-3": word[-3:],
            "pos_tag": pos_tags[i],
        }
        if i > 0:  # Previous word features
            word_features.update({
                "-1:word": sentence[i - 1],
                "-1:pos": pos_tags[i - 1],
            })
        else:
            word_features["BOS"] = True  # Beginning of Sentence

        if i < len(sentence) - 1:  # Next word features
            word_features.update({
                "+1:word": sentence[i + 1],
                "+1:pos": pos_tags[i + 1],
            })
        else:
            word_features["EOS"] = True  # End of Sentence

        features.append(word_features)
    return features

# Extract features for train and validation sets
train_features = [extract_features(sent, pos) for sent, pos in zip(train_df["tokens"], train_df["pos_tags"])]
val_features = [extract_features(sent, pos) for sent, pos in zip(val_df["tokens"], val_df["pos_tags"])]
test_features = [extract_features(sent, pos) for sent, pos in zip(test_df["tokens"], test_df["pos_tags"])]

print("Feature extraction completed!")

Feature extraction completed!


In [12]:
#Check feature extraction work correctly or not
print(len(train_features), len(train_df))
print(len(train_features[0]))

2000 2000
8


In [9]:
# Check first token of first sentence
print(train_features[0][0])

{'word': 'MRR', 'is_upper': True, 'is_title': False, 'is_digit': False, 'prefix-3': 'MRR', 'suffix-3': 'MRR', 'pos_tag': 'NOUN', 'BOS': True, '+1:word': 'mortality', '+1:pos': 'NOUN'}


In [10]:
print(train_df["tokens"][0])  # The original sentence
print(train_df["pos_tags"][0])  # The original pos_tags

['MRR', 'mortality', 'rate', 'ratio', 'TBI', 'traumatic', 'brain', 'injury']
['NOUN', 'NOUN', 'NOUN', 'NOUN', 'PROPN', 'ADJ', 'NOUN', 'NOUN']


If you are going to continue in this file please dont run the following code.

In [13]:
import joblib

# Save files locally in Colab
joblib.dump(train_features, "train_features.pkl")
joblib.dump(val_features, "val_features.pkl")
joblib.dump(test_features, "test_features.pkl")

print("Saved .pkl files!")

Saved .pkl files!
